In [2]:
with open('data/Test_second/epigraphic_corpus-2.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

In [4]:
lines[0]

'<doc id="file32708451" filename="Epigrafika" parent_folder="web_wikipedia" url="https://uk.wikipedia.org/wiki/%D0%95%D0%BF%D1%96%D0%B3%D1%80%D0%B0%D1%84%D1%96%D0%BA%D0%B0">\n'

In [14]:
def split_txt_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    total_lines = len(lines)
    
    lines_per_part = total_lines // 40

    remainder = total_lines % 40
    
    output_dir = "data/Test_second"
    
    import os
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    start = 0
    for i in range(40):
        if i < remainder:
            lines_in_part = lines_per_part + 1
        else:
            lines_in_part = lines_per_part
        
        end = start + lines_in_part
        part_lines = lines[start:end]

        part_file_path = os.path.join(output_dir, f"part_{i + 1}.txt")
        with open(part_file_path, 'w', encoding='utf-8') as part_file:
            part_file.writelines(part_lines)

        start = end
    
    print(f"Original file has been seperated to 40 parts, and saved in {output_dir}.")

# 使用示例
split_txt_file('data/Test_second/epigraphic_corpus-2.txt')


Original file has been seperated to 20 parts, and saved in data/Test_second.


In [17]:
import spacy
model_path = 'model/first_model'
nlp = spacy.load(model_path)

In [16]:
from tqdm import tqdm
import spacy
import pandas as pd

# Load the spaCy model
nlp = spacy.load('uk_core_news_lg')

# Assume 'annotations' DataFrame is loaded and combined
# Tokenize keywords in advance
annotation_path = 'data/annotation/annotation.csv'
annotations = pd.read_csv(annotation_path)
annotations['keyword_tokens'] = annotations['keywords'].apply(lambda kw: [token.text for token in nlp(kw)])

# List of file paths
files = [
    "data/Test_second/part_1.txt",
    "data/Test_second/part_2.txt",
    "data/Test_second/part_3.txt",
    "data/Test_second/part_4.txt",
    "data/Test_second/part_5.txt",
    "data/Test_second/part_6.txt",
    "data/Test_second/part_7.txt",
    "data/Test_second/part_8.txt",
    "data/Test_second/part_9.txt",
    "data/Test_second/part_10.txt",
    "data/Test_second/part_11.txt",
    "data/Test_second/part_12.txt",
    "data/Test_second/part_13.txt",
    "data/Test_second/part_14.txt",
    "data/Test_second/part_15.txt",
    "data/Test_second/part_16.txt",
    "data/Test_second/part_17.txt",
    "data/Test_second/part_18.txt",
    "data/Test_second/part_19.txt",
    "data/Test_second/part_20.txt",
    "data/Test_second/part_21.txt",
    "data/Test_second/part_22.txt",
    "data/Test_second/part_23.txt",
    "data/Test_second/part_24.txt",
    "data/Test_second/part_25.txt",
    "data/Test_second/part_26.txt",
    "data/Test_second/part_27.txt",
    "data/Test_second/part_28.txt",
    "data/Test_second/part_29.txt",
    "data/Test_second/part_30.txt",
    "data/Test_second/part_31.txt",
    "data/Test_second/part_32.txt",
    "data/Test_second/part_33.txt",
    "data/Test_second/part_34.txt",
    "data/Test_second/part_35.txt",
    "data/Test_second/part_36.txt",
    "data/Test_second/part_37.txt",
    "data/Test_second/part_38.txt",
    "data/Test_second/part_39.txt",
    "data/Test_second/part_40.txt",
    # Assume other files as needed
]

# Prepare to write to the TSV file
with open("data/Test_second/data/data.tsv", "w", encoding='utf-8') as f:
    # Process each file
    for file_path in tqdm(files, total=len(files)):
        with open(file_path, 'r', encoding='utf-8') as file:
            text = file.read()

        doc = nlp(text.strip())

        # Iterate over the sentences
        for sent in doc.sents:
            sent_tokens = [token.text for token in sent if token.text not in ['\t', '\n', ' ']]
            token_labels = ['O' for _ in sent_tokens]  # Initialize labels as 'O' for each token

            # Check for matching keyword token sequences
            for _, row in annotations.iterrows():
                keyword_tokens = row['keyword_tokens']
                for i in range(len(sent_tokens) - len(keyword_tokens) + 1):
                    if sent_tokens[i:i + len(keyword_tokens)] == keyword_tokens:
                        # Match found, apply BIO tagging
                        if all(label == 'O' for label in token_labels[i:i+len(keyword_tokens)]):  # Ensure not to overwrite existing tags
                            token_labels[i] = f'B-{row["type"]}'
                            for j in range(1, len(keyword_tokens)):
                                token_labels[i + j] = f'I-{row["type"]}'

            # Write tokens and tags to file, with newline after each sentence
            for token, label in zip(sent_tokens, token_labels):
                f.write(f"{token}\t{label}\n")
            f.write("\n")  # Newline after each sentence to separate them


100%|██████████████████████████████████████████████████████████████████████████████████| 40/40 [21:58<00:00, 32.96s/it]
